In [1]:
 !pip install fpdf2
 

In [2]:
!wget https://gitlab.com/didovgopoly/taste_it/-/raw/master/src/model/save_to_pdf.py

--2020-12-10 09:23:53--  https://gitlab.com/didovgopoly/taste_it/-/raw/master/src/model/save_to_pdf.py
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘save_to_pdf.py.1’

save_to_pdf.py.1        [ <=>                ]   2.26K  --.-KB/s    in 0s      

2020-12-10 09:23:53 (12.4 MB/s) - ‘save_to_pdf.py.1’ saved [2312]



In [3]:
!nvidia-smi

Thu Dec 10 09:23:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    36W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from pathlib import Path
import pickle
import random
from collections import OrderedDict
import os

from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision import models
import torch.nn.functional as F
import save_to_pdf
import importlib
from importlib import reload

reload(save_to_pdf)

<module 'save_to_pdf' from '/content/save_to_pdf.py'>

In [5]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/text2image'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%%time
os.makedirs('data', exist_ok=True)
os.makedirs('data/embeddings', exist_ok=True)
os.makedirs('/content/drive/My Drive/text2image/fake_images', exist_ok=True)
os.makedirs('/content/drive/My Drive/text2image/fake_images/tmp', exist_ok=True)
os.makedirs('/content/drive/My Drive/text2image/models', exist_ok=True)
if not os.path.exists('GDriveDL.py'):
    !wget https://raw.githubusercontent.com/matthuisman/gdrivedl/master/gdrivedl.py -O GDriveDL.py


if not os.path.exists('data/eda_povar.zip'):
    !python GDriveDL.py https://drive.google.com/file/d/15YnVwez09ikuRCZMcKzL2CdThitjXB3e/view?usp=sharing data
    !unzip -q data/eda_povar.zip -d data

# if not os.path.exists('df_gan.zip'):        
#     !python GDriveDL.py https://drive.google.com/open?id=1KKAqwSbHd-_qMpOAjYdbBRCh4M-HbRTH .
#     !unzip -q df_gan.zip

CPU times: user 303 µs, sys: 715 µs, total: 1.02 ms
Wall time: 4.58 ms


In [7]:
def load_resnet(resnet_path, food_space_dim=512):
    resnet_fe = models.resnet50(pretrained=False)
    num_ftrs = resnet_fe.fc.in_features
    resnet_fe.fc = nn.Linear(num_ftrs, food_space_dim) # Чтобы загрузить веса обученной ранее модели модели
    resnet_fe.load_state_dict(torch.load(resnet_path))
    return resnet_fe

resnet_ingredients_path = Path('models/best_resnet_title_ingr.pth')
resnet_steps_path = Path('models/best_resnet_steps.pth')

os.makedirs('models', exist_ok=True)

if not os.path.exists(resnet_ingredients_path): 
    !python GDriveDL.py https://drive.google.com/file/d/1udslmO77CcvKKnfY7y-NkbleqC0QsfOz/view?usp=sharing models

if not os.path.exists(resnet_steps_path): 
    !python GDriveDL.py https://drive.google.com/file/d/1qjQW3MUfOc46bEYPIzeFT1wXqJfEIEPJ/view?usp=sharing models

resnet_ingr = load_resnet(resnet_ingredients_path, 256)
resnet_steps = load_resnet(resnet_steps_path, 256)

### 1) Dataset

In [8]:
class RecipeDataset(data.Dataset):
    def __init__(self, 
                 data_dir='data', 
                 csv_filename='dataset.csv',
                 use_last_image=False,
                 base_size=64,
                 transform=None, target_transform=None):
        self.transform = transform
        self.norm = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        self.target_transform = target_transform
        
        self.data_dir = Path(data_dir)
        self.csv_filename = csv_filename
        self.use_last_image = use_last_image
        
        self.data = pd.read_csv(self.data_dir / self.csv_filename, usecols=['id', 'images', 'title', 'source'])
        self.ids = self.data['id'].values
        self.source = self.data['source'].values
        self.images = self.data['images'].values
        self.titles = self.data['title'].values
        
        self.embeddings_ingredients, self.embeddings_steps = self.load_embeddings()

    
    def load_embeddings(self):

        embeddings_ingredients = []
        embeddings_steps = []

        for idx, source in zip(self.ids, self.source):
            emb_ingr = np.load(f'{self.data_dir}/embeddings/lstm_title_ingredients_256/{source}_{idx}.npz')['arr_0']
            emb_step = np.load(f'{self.data_dir}/embeddings/lstm_steps_256/{source}_{idx}.npz')['arr_0']
            # title_emb = np.load(f'{self.data_dir}/embeddings/title/{idx}.npz')['arr_0']
            # ingredients_emb = np.load(f'{self.data_dir}/embeddings/ingredients/{idx}.npz')['arr_0']
            # steps_emb = np.load(f'{self.data_dir}/embeddings/steps/{idx}.npz')['arr_0']
            # all_recipe_emb = np.load(f'{self.data_dir}/embeddings/all_recipe/{idx}.npz')['arr_0']

            embeddings_ingredients.append(
                # np.concatenate([title_emb, ingredients_emb], axis=1)
                [emb_ingr]
            )

            embeddings_steps.append(
                # np.concatenate([title_emb, ingredients_emb], axis=1)
                [emb_step]
            )

        return np.array(embeddings_ingredients), np.array(embeddings_steps)


    def __getitem__(self, idx):
        #
        image_pathes = self.images[idx].split('|')
        if self.use_last_image:
            img_path = image_pathes[-1]
        else:
            img_path = random.choice(image_pathes)
       
        # Загружаем изображение
        img = Image.open(self.data_dir / img_path).convert('RGB')
        width, height = img.size
        if self.transform is not None:
            img = self.transform(img)
        imgs = []
        imgs.append(self.norm(img))
        
        # ИСПРАВИТЬ - временно для того чтобы проверить работоспособность в целом
        emb_ingr = self.embeddings_ingredients[idx, :][0]
        emb_steps = self.embeddings_steps[idx, :][0]
        
        return imgs, emb_ingr, emb_steps, self.titles[idx]
    
    def __len__(self):
        return len(self.ids)

In [9]:
EMB_SIZE = 512 # 768 * 2

class NetG(nn.Module):
    def __init__(self, ngf=64, nz=100):
        super(NetG, self).__init__()

        # self.fc_embedding = nn.Linear(768, 256)

        self.ngf = ngf

        # layer1输入的是一个100x1x1的随机噪声, 输出尺寸(ngf*8)x4x4
        self.fc = nn.Linear(nz, ngf*8*4*4)
        self.block0 = G_Block(ngf * 8, ngf * 8)#4x4
        self.block1 = G_Block(ngf * 8, ngf * 8)#4x4
        self.block2 = G_Block(ngf * 8, ngf * 8)#8x8
        self.block3 = G_Block(ngf * 8, ngf * 8)#16x16
        self.block4 = G_Block(ngf * 8, ngf * 4)#32x32
        # self.block5 = G_Block(ngf * 4, ngf * 2)#64x64
        # self.block6 = G_Block(ngf * 2, ngf * 1)#128x128
        self.block5 = G_Block(ngf * 4, ngf * 1)

        self.conv_img = nn.Sequential(
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ngf, 3, 3, 1, 1),
            nn.Tanh(),
        )

    def forward(self, x, c):

        # c = self.fc_embedding(c)

        out = self.fc(x)
        out = out.view(x.size(0), 8*self.ngf, 4, 4)
        out = self.block0(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block1(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block2(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block3(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block4(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block5(out,c)
        # out = F.interpolate(out, scale_factor=2)
        # out = self.block6(out,c)

        out = self.conv_img(out)

        return out


class G_Block(nn.Module):

    def __init__(self, in_ch, out_ch):
        super(G_Block, self).__init__()

        self.learnable_sc = in_ch != out_ch 
        self.c1 = nn.Conv2d(in_ch, out_ch, 3, 1, 1)
        self.c2 = nn.Conv2d(out_ch, out_ch, 3, 1, 1)
        self.affine0 = affine(in_ch)
        self.affine1 = affine(in_ch)
        self.affine2 = affine(out_ch)
        self.affine3 = affine(out_ch)
        self.gamma = nn.Parameter(torch.zeros(1))
        if self.learnable_sc:
            self.c_sc = nn.Conv2d(in_ch,out_ch, 1, stride=1, padding=0)

    def forward(self, x, y=None):
        return self.shortcut(x) + self.gamma * self.residual(x, y)

    def shortcut(self, x):
        if self.learnable_sc:
            x = self.c_sc(x)
        return x

    def residual(self, x, y=None):
        h = self.affine0(x, y)
        h = nn.LeakyReLU(0.2,inplace=True)(h)
        h = self.affine1(h, y)
        h = nn.LeakyReLU(0.2,inplace=True)(h)
        h = self.c1(h)
        
        h = self.affine2(h, y)
        h = nn.LeakyReLU(0.2,inplace=True)(h)
        h = self.affine3(h, y)
        h = nn.LeakyReLU(0.2,inplace=True)(h)
        return self.c2(h)



class affine(nn.Module):

    def __init__(self, num_features):
        super(affine, self).__init__()

        self.fc_gamma = nn.Sequential(OrderedDict([
            ('linear1',nn.Linear(EMB_SIZE, 256)),
            ('relu1',nn.ReLU(inplace=True)),
            ('linear2',nn.Linear(256, num_features)),
            ]))
        self.fc_beta = nn.Sequential(OrderedDict([
            ('linear1',nn.Linear(EMB_SIZE, 256)),
            ('relu1',nn.ReLU(inplace=True)),
            ('linear2',nn.Linear(256, num_features)),
            ]))
        self._initialize()

    def _initialize(self):
        nn.init.zeros_(self.fc_gamma.linear2.weight.data)
        nn.init.ones_(self.fc_gamma.linear2.bias.data)
        nn.init.zeros_(self.fc_beta.linear2.weight.data)
        nn.init.zeros_(self.fc_beta.linear2.bias.data)

    def forward(self, x, y=None):

        weight = self.fc_gamma(y)
        bias = self.fc_beta(y)        

        if weight.dim() == 1:
            weight = weight.unsqueeze(0)
        if bias.dim() == 1:
            bias = bias.unsqueeze(0)

        size = x.size()
        weight = weight.unsqueeze(-1).unsqueeze(-1).expand(size)
        bias = bias.unsqueeze(-1).unsqueeze(-1).expand(size)
        return weight * x + bias


class D_GET_LOGITS(nn.Module):
    def __init__(self, ndf):
        super(D_GET_LOGITS, self).__init__()
        self.df_dim = ndf
        # self.fc_embedding = nn.Linear(768, 256)

        self.joint_conv = nn.Sequential(
            nn.Conv2d(ndf * 16 + EMB_SIZE, ndf * 2, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf * 2, 1, 4, 1, 0, bias=False),
        )

    def forward(self, out, y):

        # y = self.fc_embedding(y)
        
        y = y.view(-1, EMB_SIZE, 1, 1)
        y = y.repeat(1, 1, 4, 4)
        h_c_code = torch.cat((out, y), 1)
        out = self.joint_conv(h_c_code)
        return out


class NetD(nn.Module):
    def __init__(self, ndf):
        super(NetD, self).__init__()

        self.conv_img = nn.Conv2d(3, ndf, 3, 1, 1)#128
        self.block0 = resD(ndf * 1, ndf * 2)#64
        self.block1 = resD(ndf * 2, ndf * 4)#32
        self.block2 = resD(ndf * 4, ndf * 8)#16
        self.block3 = resD(ndf * 8, ndf * 16)#8
        self.block4 = resD(ndf * 16, ndf * 16)#4
        self.block5 = resD(ndf * 16, ndf * 16)#4

        self.COND_DNET = D_GET_LOGITS(ndf)

    def forward(self,x):
        
        out = self.conv_img(x)
        out = self.block0(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        # out = self.block5(out)


        return out


class resD(nn.Module):
    def __init__(self, fin, fout, downsample=True):
        super().__init__()
        self.downsample = downsample
        self.learned_shortcut = (fin != fout)
        self.conv_r = nn.Sequential(
            nn.Conv2d(fin, fout, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(fout, fout, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.conv_s = nn.Conv2d(fin,fout, 1, stride=1, padding=0)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x, c=None):
        return self.shortcut(x)+self.gamma*self.residual(x)

    def shortcut(self, x):
        if self.learned_shortcut:
            x = self.conv_s(x)
        if self.downsample:
            return F.avg_pool2d(x, 2)
        return x

    def residual(self, x):
        return self.conv_r(x)

## 2) Обучение

In [10]:
seed = 100
print("seed now is : ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
cudnn.benchmark = True

seed now is :  100


In [11]:
device

device(type='cuda')

#### Config:

In [12]:
config = {}

config['MODEL_NAME'] = 'eda_ru_df_gan_lstm_cycle_loss_ingr_steps_128'

config['CUDA'] = True
config['WORKERS'] = 4

config['loss'] = 'hinge'
config['BASE_SIZE'] = 64

# config['DATASET'] = {}
# config['DATASET']['EMBEDDINGS_FILE'] = 'RuBERT_mean_embeddings_long'
# config['DATASET']['EMBEDDINGS_TYPE'] = 'whole_recipe'


config['IMAGE_SIZE'] = 128
config['TRAIN'] = {}
config['TRAIN']['BATCH_SIZE'] = 32
config['TRAIN']['MAX_EPOCH'] = 512
config['TRAIN']['SNAPSHOT_INTERVAL'] = 2000
config['TRAIN']['DISCRIMINATOR_LR'] = 2e-4
config['TRAIN']['GENERATOR_LR'] = 2e-4
config['TRAIN']['ENCODER_LR'] = 2e-4
# config['TRAIN']['RNN_GRAD_CLIP'] = 0.25
# config['TRAIN']['FLAG'] = True
# config['TRAIN']['NET_E'] = ''
# config['TRAIN']['NET_G'] = ''
# config['TRAIN']['B_NET_D'] = True
config['TRAIN']['NF'] = 32
config['TRAIN']['SMOOTH'] = {}
config['TRAIN']['SMOOTH']['GAMMA1'] = 5.0
config['TRAIN']['SMOOTH']['GAMMA1'] = 10.0
config['TRAIN']['SMOOTH']['GAMMA1'] = 5.0
config['TRAIN']['SMOOTH']['GAMMA1'] = 1.0
config['TRAIN']['CYCLE_LOSS_COEFF'] = 1.0

config['GAN'] = {}
config['GAN']['DF_DIM'] = 64
config['GAN']['GF_DIM'] = 128
config['GAN']['Z_DIM'] = 100
config['GAN']['CONDITION_DIM'] = 100
config['GAN']['R_NUM'] = 2
config['GAN']['B_ATTENTION'] = True
config['GAN']['B_DCGAN'] = True

In [13]:
# dataset and dataloader

# image_transform = transforms.Compose([
#     transforms.Resize(int(config['IMAGE_SIZE'] * 76 / 64)),
#     transforms.RandomCrop(config['IMAGE_SIZE']),
#     transforms.RandomHorizontalFlip()])

image_transform = transforms.Compose([
    transforms.Resize(int(config['IMAGE_SIZE']))
    ])
   
dataset = RecipeDataset(transform=image_transform)

print(len(dataset))
assert dataset
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=config['TRAIN']['BATCH_SIZE'], drop_last=True,
    shuffle=True, num_workers=config['WORKERS'])

98526


In [14]:
def prepare_data(data):
    imgs, embds_ingr, embds_steps, titles = data

    real_imgs = []
    for i in range(len(imgs)):
        if config['CUDA']:
            real_imgs.append(Variable(imgs[i]).cuda())
        else:
            real_imgs.append(Variable(imgs[i]))

    if config['CUDA']:
        embds_ingr = Variable(embds_ingr).cuda()
        embds_steps = Variable(embds_steps).cuda()
    else:
        embds_ingr = Variable(embds_ingr)
        embds_steps = Variable(embds_steps)

    return [real_imgs, embds_ingr, embds_steps, titles]

In [15]:
resnet_ingr.cuda()
resnet_ingr.eval()
resnet_steps.cuda()
resnet_steps.eval()

image_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [22]:
def train(dataloader, netG, netD, optimizerG, optimizerD, state_epoch, batch_size, device):
    fake_images = Path(f'{drive_path}/fake_images')
    fake_images.mkdir(exist_ok=True)
    fake_images = fake_images / f'{config["MODEL_NAME"]}'
    fake_images.mkdir(exist_ok=True)
    (fake_images / 'tmp').mkdir(exist_ok=True)

    for epoch in range(state_epoch + 1, config['TRAIN']['MAX_EPOCH'] + 1):
        for step, data in enumerate(dataloader, 0):
            imags, emb_1, emb_2, titles = prepare_data(data)
            sent_emb = torch.hstack([emb_1, emb_2])
#             hidden = text_encoder.init_hidden(batch_size)
#             # words_embs: batch_size x nef x seq_len
#             # sent_emb: batch_size x nef
#             words_embs, sent_emb = text_encoder(captions, cap_lens, hidden)
#             words_embs, sent_emb = words_embs.detach(), sent_emb.detach()

            imgs=imags[0].to(device)
            real_features = netD(imgs)
            # print(imgs.shape)
            # print(real_features.shape)
            # print(sent_emb.shape)
            output = netD.COND_DNET(real_features, sent_emb)
            errD_real = torch.nn.ReLU()(1.0 - output).mean()


            output = netD.COND_DNET(real_features[:(batch_size - 1)], sent_emb[1:batch_size])
            errD_mismatch = torch.nn.ReLU()(1.0 + output).mean()

            # synthesize fake images
            noise = torch.randn(batch_size, 100)
            noise=noise.to(device)
            fake = netG(noise,sent_emb)  
            
            # G does not need update with D
            fake_features = netD(fake.detach()) 

            errD_fake = netD.COND_DNET(fake_features,sent_emb)
            errD_fake = torch.nn.ReLU()(1.0 + errD_fake).mean()          

            errD = errD_real + (errD_fake + errD_mismatch)/2.0
            optimizerD.zero_grad()
            optimizerG.zero_grad()
            errD.backward()
            optimizerD.step()

            #MA-GP
            interpolated = (imgs.data).requires_grad_(True)
            sent_inter = (sent_emb.data).requires_grad_(True)
            features = netD(interpolated)
            out = netD.COND_DNET(features,sent_inter)
            grads = torch.autograd.grad(outputs=out,
                                    inputs=(interpolated,sent_inter),
                                    grad_outputs=torch.ones(out.size()).cuda(),
                                    retain_graph=True,
                                    create_graph=True,
                                    only_inputs=True)
            grad0 = grads[0].view(grads[0].size(0), -1)
            grad1 = grads[1].view(grads[1].size(0), -1)
            grad = torch.cat((grad0,grad1),dim=1)                        
            grad_l2norm = torch.sqrt(torch.sum(grad ** 2, dim=1))
            d_loss_gp = torch.mean((grad_l2norm) ** 6)
            d_loss = 2.0 * d_loss_gp
            optimizerD.zero_grad()
            optimizerG.zero_grad()
            d_loss.backward()
            optimizerD.step()
            
            # update G
            features = netD(fake)
            output = netD.COND_DNET(features,sent_emb)
            
            errG = 0

            # Cycle loss
            if config['TRAIN']['CYCLE_LOSS_COEFF'] > 0:
                resized_fake = image_transforms(fake)
                fake_feats_ingr = resnet_ingr(resized_fake)
                fake_feats_steps = resnet_steps(resized_fake)
                err_cycle_ingr = F.cosine_similarity(fake_feats_ingr, emb_1).mean()
                err_cycle_steps = F.cosine_similarity(fake_feats_steps, emb_2).mean()
                errG = -err_cycle_ingr * config['TRAIN']['CYCLE_LOSS_COEFF'] * 0.5
                errG = -err_cycle_steps * config['TRAIN']['CYCLE_LOSS_COEFF'] * 0.5

            errG -= output.mean()
            optimizerG.zero_grad()
            optimizerD.zero_grad()
            errG.backward()
            optimizerG.step()

            if step % 1000 == 0:
                print('[%d/%d][%d/%d] Loss_D: %.3f Loss_G %.3f Ingred_similarity: %.3f Steps_similarity: %.3f'
                    % (epoch, config['TRAIN']['MAX_EPOCH'], step, len(dataloader), errD.item(), errG.item(), err_cycle_ingr.item(), err_cycle_steps.item()))

                save_to_pdf.save_pdf_images(fake.data, titles, str(fake_images), f'{config["MODEL_NAME"]}_{epoch}_{step}')

        if epoch % 1 == 0:
            path_netG = f'{drive_path}/models/{config["MODEL_NAME"]}_netG.pth' 
            path_netD = f'{drive_path}/models/{config["MODEL_NAME"]}_netD.pth' 
            torch.save(netG.state_dict(), path_netG)
            torch.save(netD.state_dict(), path_netD)      

In [23]:
netG = NetG(ngf=config['TRAIN']['NF'], nz=100)
netD = NetD(ndf=config['TRAIN']['NF'])
state_epoch=0


# load_epoch = 94
# path_netG = f'{drive_path}/models/{config["MODEL_NAME"]}_netG_{load_epoch}.pth' 
# path_netD = f'{drive_path}/models/{config["MODEL_NAME"]}_netD_{load_epoch}.pth' 
# netG.load_state_dict(torch.load(path_netG))
# netD.load_state_dict(torch.load(path_netD))
# state_epoch = load_epoch


netG.to(device)
netD.to(device)
optimizerG = torch.optim.Adam(netG.parameters(), lr=0.0001, betas=(0.0, 0.9))
optimizerD = torch.optim.Adam(netD.parameters(), lr=0.0004, betas=(0.0, 0.9))

In [ ]:
train(dataloader, netG, netD, optimizerG, optimizerD, state_epoch, config['TRAIN']['BATCH_SIZE'], device)

[1/512][0/3078] Loss_D: 1.999 Loss_G 0.093 Ingred_similarity: 0.111 Steps_similarity: -0.198
[1/512][1000/3078] Loss_D: 1.347 Loss_G 1.781 Ingred_similarity: 0.225 Steps_similarity: 0.326
[1/512][2000/3078] Loss_D: 1.645 Loss_G 0.399 Ingred_similarity: 0.084 Steps_similarity: 0.854
[1/512][3000/3078] Loss_D: 1.857 Loss_G -0.208 Ingred_similarity: 0.035 Steps_similarity: 0.811
[2/512][0/3078] Loss_D: 1.718 Loss_G 0.078 Ingred_similarity: 0.234 Steps_similarity: 0.890
[2/512][1000/3078] Loss_D: 1.754 Loss_G 1.052 Ingred_similarity: 0.065 Steps_similarity: 0.919
[2/512][2000/3078] Loss_D: 2.127 Loss_G 1.231 Ingred_similarity: 0.207 Steps_similarity: 0.825
[2/512][3000/3078] Loss_D: 1.807 Loss_G 1.318 Ingred_similarity: 0.064 Steps_similarity: 0.763
[3/512][0/3078] Loss_D: 1.578 Loss_G 0.508 Ingred_similarity: -0.101 Steps_similarity: 0.820
[3/512][1000/3078] Loss_D: 1.558 Loss_G 1.202 Ingred_similarity: 0.150 Steps_similarity: 0.861
